In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
root_dir = '/kaggle/input/ai-vs-human-generated-dataset/'
train_csv = pd.read_csv("/kaggle/input/ai-vs-human-generated-dataset/train.csv")
test_csv = pd.read_csv("/kaggle/input/ai-vs-human-generated-dataset/test.csv")

In [3]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(train_csv, test_size=0.2, random_state=42, stratify=train_csv['label'])
train_df.head()

,Unnamed: 0,file_name,label
23023,23023,train_data/061ae7c248dc4ee783d75147c3d7cb81.jpg,0
65033,65033,train_data/263b231189294ce4966559abe6977152.jpg,0
41549,41549,train_data/ac4e1f53a1b24dcb9a825e6850a68560.jpg,0
61564,61564,train_data/8be9a8e197294052951caefa11d38a1b.jpg,1
72781,72781,train_data/28827a35b71e48ed8981add25fc948c9.jpg,0


In [4]:
from torch.utils.data import Dataset
import os 
from PIL import Image

class CustomDataset(Dataset):
    def __init__(self, root_dir= None, dataframe= None, transform = None):
        super(CustomDataset, self).__init__()
        self.dataframe = dataframe
        self.transform = transform
        self.root_dir = root_dir

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        image_path = os.path.join(self.root_dir, self.dataframe.iloc[idx]["file_name"])
        image = Image.open(image_path).convert("RGB")
        label = int(self.dataframe.iloc[idx]["label"])

        if self.transform:
            image = self.transform(image)

        return image, label

In [5]:
from torchvision.transforms import transforms

train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],   # Mean for ImageNet
                         [0.229, 0.224, 0.225])   # Std for ImageNet
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

train_dataset = CustomDataset(
    root_dir= root_dir, 
    dataframe= train_df, 
    transform = train_transforms
)

val_dataset = CustomDataset(
    root_dir= root_dir, 
    dataframe= val_df, 
    transform = val_transforms
)

In [6]:
from torch.utils.data import DataLoader

batch_size = 32
train_dataloader = DataLoader(train_dataset, 
                            batch_size=batch_size,
                            shuffle=True,
                            num_workers=2)

val_dataloader = DataLoader(val_dataset, 
                            batch_size=batch_size,
                            shuffle=False,
                            num_workers=2)

In [7]:
import torch 
from torch import nn
from torch import optim

In [8]:
def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer):
    
    model.train()
    train_loss, train_acc = 0, 0

    for batch, (x,y) in enumerate(dataloader):
        x, y = x.to(device), y.to(device)

        y_pred = model(x)

        loss = loss_fn(y_pred, y)
        train_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()

        optimizer.step()

        y_pred_class = torch.argmax(torch.softmax(y_pred, dim =1), dim = 1)
        train_acc += (y_pred_class == y).sum().item() / len(y_pred_class)


    # avg train loss and acc
    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)

    return train_loss, train_acc

def val_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module):
    
    model.eval()

    val_loss, val_acc = 0,0

    with torch.inference_mode():
        for batch, (x,y) in enumerate(dataloader):
            x, y = x.to(device), y.to(device)

            val_y_pred = model(x)

            loss = loss_fn(val_y_pred, y)
            test_loss += loss

            val_y_pred_class = torch.argmax(torch.softmax(val_y_pred, dim = 1), dim = 1)
            val_acc += (val_y_pred_class == y).sum().item() / len(val_y_pred_class)

    val_loss = val_loss / len(dataloader)
    val_acc = val_acc / len(dataloader)

    return val_loss, val_acc

# from tqdm import tqdm 
# start  the timer 
from timeit import default_timer as timer 

def train(model:nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          val_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int = 50):
    
    # create an empty results dir
    results = {"train_loss":[],
               "val_loss":[],
               "train_acc":[],
               "val_acc":[]}
    
    # loop for training an testing
    for epoch in range(epochs):
        epoch_start_time = timer()
        train_loss, train_acc = train_step(model = model,
                                           dataloader = train_dataloader,
                                           loss_fn = loss_fn,
                                           optimizer = optimizer)

        val_loss, val_acc = val_step(model = model,
                                        dataloader = val_dataloader,
                                        loss_fn = loss_fn)
        
        epoch_end_time = timer()
        
        print(
            f"epochs: {epoch+1} | train_loss: {train_loss} | train_acc: {train_acc} | test_loss: {val_loss} | test_acc: {val_acc}"
            )
        
        # update the results dictionary
        results["train_loss"].append(train_loss.item() if isinstance(train_loss, torch.Tensor) else train_loss)
        results["train_acc"].append(train_acc.item() if isinstance(train_acc, torch.Tensor) else train_acc)
        results["val_loss"].append(val_loss.item() if isinstance(val_loss, torch.Tensor) else val_loss)
        results["val_acc"].append(val_acc.item() if isinstance(val_acc, torch.Tensor) else val_acc)

    return results

In [ ]:
######## train and val model ########
from torchvision import models

torch.manual_seed(42)
torch.cuda.manual_seed(42)

num_epochs = 25
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load a pretrained ResNeXt model
model = models.resnext50_32x4d(pretrained=True) # or resnext101_32x8d
# Freeze early layers if you want (optional)
for param in model.parameters():
    param.requires_grad = True  # By default, we unfreeze everything for fine-tuning

# Replace the final classification layer
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)  # 2 classes
model = model.to(device)
# Define loss and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

start_time = timer()

# Training function
model_results = train(model=model,
                      train_dataloader=train_dataloader,
                      val_dataloader=val_dataloader,
                      loss_fn=loss_fn,
                      optimizer=optimizer,
                      epochs=num_epochs)

end_time = timer()

print(f"total training time: {end_time - start_time:.3f} seconds")


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNeXt50_32X4D_Weights.IMAGENET1K_V1`. You can also use `weights=ResNeXt50_32X4D_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth" to /root/.cache/torch/hub/checkpoints/resnext50_32x4d-7cdf4587.pth
100%|██████████| 95.8M/95.8M [00:00<00:00, 172MB/s] 


In [ ]:
class TestDataset(Dataset):
    def __init__(self, df,root_dir, transform=None):
        self.df = df
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_name = os.path.join(self.root_dir,self.df.iloc[idx]['id'])
        image = Image.open(file_name).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        return image
        
test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

test_dataset = TestDataset(test_csv, transform=test_transforms)
test_loader = DataLoader(test_dataset, 
                         batch_size=32, 
                         num_workers=2,
                         shuffle=False)

model.eval()
all_preds = []
with torch.no_grad():
    for images in test_loader:
        images = images.to(device)
        outputs = model(images)  # shape [batch_size, 2] if using nn.CrossEntropyLoss
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy().tolist())

submission_df = pd.DataFrame({
    'id': test_csv['id'],      
    'label': all_preds       
})

submission_df.to_csv('submission.csv', index=False)
